In [ ]:

import numpy as np
import pandas as pd 

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
df_full=pd.read_csv('/kaggle/input/zomato-bangalore-restaurants/zomato.csv')

# I'm not using full Data as it taking to much time to ru the notebook, I have taken only 100 values.

In [ ]:
df=df_full.head(100)

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
df.isnull().sum()

In [ ]:
df=df.drop(['url','phone','dish_liked'],axis=1)

In [ ]:
df.nunique()

In [ ]:
df.drop_duplicates(inplace=True)

In [ ]:
df.dropna(inplace=True)

In [ ]:
d={'Yes':1,'No':0}
df['online_order']=df['online_order'].map(d)
df['book_table']=df['book_table'].map(d)

In [ ]:
df=df.rename(columns={'approx_cost(for two people)':'cost','listed_in(type)':'type', 'listed_in(city)':'city'})

In [ ]:
df['rate']=df['rate'].replace('NEW','0')
df['rate']=df['rate'].replace('-','0')
df['rate']=df['rate'].str.replace('/5','').astype(float)

In [ ]:
df['cost']=df['cost'].str.replace(',','').astype(int)

In [ ]:
restaurants = list(df['name'].unique())
df['Mean Rating'] = 0

for i in range(len(restaurants)):
    df['Mean Rating'][df['name'] == restaurants[i]] = df['rate'][df['name'] == restaurants[i]].mean()
    
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range = (1,5))
df[['Mean Rating']] = scaler.fit_transform(df[['Mean Rating']]).round(2)

In [ ]:
import nltk
import string
import re

In [ ]:
df['reviews_list']=df['reviews_list'].astype(str)

In [ ]:
def text_lowercase(text):
    return text.lower()

df['reviews_list']=text_lowercase(df['reviews_list'].str)

In [ ]:
def remove_punctuation(text):
    translator = str.maketrans('', '', string.punctuation)
    return text.translate(translator)
  

df['reviews_list']=remove_punctuation(df['reviews_list'].str)

In [ ]:
def remove_numbers(text):
    result = re.sub(r'\d+', '', text)
    return result

df['reviews_list']=df['reviews_list'].apply(remove_numbers)

In [ ]:
df['reviews_list']=df['reviews_list'].str.replace('rated  ratedn','')

In [ ]:
def remove_whitespace(text):
    return  " ".join(text.split())
  

df['reviews_list']=df['reviews_list'].apply(remove_whitespace)

In [ ]:
nltk.download('stopwords')
nltk.download('punkt')

In [ ]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
  
def remove_stopwords(text):
    stop_words = set(stopwords.words("english"))
    word_tokens = word_tokenize(text)
    return  " ".join([word for word in word_tokens if word not in stop_words])


df['reviews_list']=df['reviews_list'].apply(remove_stopwords)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(df['reviews_list'])
#cosine_similarities = linear_kernel(X, X)
cos_sim = cosine_similarity(X, X)

In [ ]:
def recommend(name, cosine_similarities = cos_sim):

    recommend_restaurant = []
    idx = df[df == name].index[0]
    score_series = pd.Series(cosine_similarities[idx]).sort_values(ascending=False)
    top10_indexes = list(score_series.iloc[0:11].index)

    for each in top10_indexes:
        recommend_restaurant.append(list(df.index)[each])

    df_new = pd.DataFrame(columns=['cuisines', 'Mean Rating', 'cost'])

    for each in recommend_restaurant:
        df_new = df_new.append(pd.DataFrame(df[['cuisines','Mean Rating', 'cost']][df.index == each].sample()))

    df_new = df_new.drop_duplicates(subset=['cuisines','Mean Rating', 'cost'], keep=False)
    df_new = df_new.sort_values(by='Mean Rating', ascending=False).head(10)
    
    print('Top %s Resturents like %s with similar reviews: ' % (str(len(df_new)), name))
    
    return df_new
recommend('Pai Vihar')